 
# Train an Object Detection Model using Transfer Learning with SSD MobileNet V2

This notebook demonstrates:
1. Loading a pre-trained SSD MobileNet V2 model.
2. Customizing the model for a specific object detection task.
3. Training and evaluating the model.
4. Exporting the model as a TensorFlow Lite (`.tflite`) file.


In [ ]:

# Install necessary libraries
!pip install tensorflow tensorflow_hub tensorflow-datasets
!pip install tf_slim


In [ ]:

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


### Load and Prepare the Dataset

In [ ]:

# Example: Use TensorFlow Datasets for training (e.g., COCO dataset or your custom dataset)
import tensorflow_datasets as tfds

# Load a sample dataset
dataset_name = 'cats_vs_dogs'
data, info = tfds.load(dataset_name, as_supervised=True, with_info=True)

train_data, test_data = data['train'], data['test']

# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, (300, 300)) / 255.0  # Resize to 300x300 for SSD MobileNet
    return image, label

train_data = train_data.map(preprocess).batch(32).shuffle(1000)
test_data = test_data.map(preprocess).batch(32)


### Load Pre-trained SSD MobileNet V2 Model

In [ ]:

# Load pre-trained SSD MobileNet V2 model from TensorFlow Hub
base_model_url = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2"
model = hub.load(base_model_url)

# Example of how to fine-tune the model will depend on the specific dataset and problem
# The hub.KerasLayer can be modified for training


### Fine-tune the Model

In [ ]:

# Create a custom training head for fine-tuning
model = tf.keras.Sequential([
    hub.KerasLayer(base_model_url, trainable=False),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification, modify for multi-class
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data, epochs=5, validation_data=test_data)


### Evaluate the Model

In [ ]:

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_data)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


### Convert the Model to TensorFlow Lite

In [ ]:

# Convert the trained model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
tflite_model_file = 'ssd_mobilenet_v2.tflite'
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

print(f"Model has been successfully saved as '{tflite_model_file}'.")
